<a href="https://colab.research.google.com/github/Kashalpha/McD-nutrient-balance/blob/main/%E5%AE%8C%E5%85%A8%E6%A0%84%E9%A4%8A%E3%83%9E%E3%82%AF%E3%83%89%E3%83%8A%E3%83%AB%E3%83%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pulp

     |████████████████████████████████| 40.6MB 93kB/s 


In [1]:
import pandas as pd
import numpy as np
import pulp

In [2]:
df = pd.read_csv("マクドナルド栄養価.csv",index_col=0)

In [3]:
# 欠損値を0埋め
df = df.fillna(0)

In [4]:
# カロリーが0の商品を除く
df = df[df['エネルギー']>0]

In [5]:
# 統計情報の確認
df.describe()

,製品重量,エネルギー,タンパク質,脂質,炭水化物,ナトリウム,カリウム,カルシウム,リン,鉄,ビタミンA,ビタミンB1,ビタミンB2,ナイアシン,ビタミンC,コレステロール,食物繊維,食塩相当量
count,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000,171.000000
mean,223.461988,286.888889,10.525731,13.728070,30.301170,479.725146,280.719298,72.894737,163.339181,0.828655,48.421053,0.168363,0.185205,2.730994,6.538012,48.157895,1.328655,1.216374
std,141.917261,263.421457,12.347919,16.264256,23.833706,538.202728,360.656041,77.234026,178.778705,1.078369,114.143297,0.419882,0.338355,4.344209,20.922055,84.416221,1.907647,1.367338
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,134.500000,88.000000,0.400000,0.100000,9.650000,15.500000,29.000000,4.500000,8.500000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000
50%,197.000000,232.000000,5.200000,6.700000,31.200000,181.000000,232.000000,42.000000,132.000000,0.300000,10.000000,0.080000,0.100000,1.000000,1.000000,5.000000,0.900000,0.500000
75%,298.500000,437.500000,18.300000,23.100000,41.850000,897.000000,346.000000,134.000000,257.500000,1.300000,61.500000,0.170000,0.245000,4.100000,2.000000,63.500000,1.850000,2.300000
max,640.000000,1844.000000,60.600000,103.400000,167.700000,2346.000000,3046.000000,394.000000,1255.000000,6.200000,1049.000000,2.890000,2.940000,33.900000,172.000000,478.000000,14.600000,6.000000


In [6]:
# 栄養値の上限・下限の入力
df_max_min = pd.read_csv("1日分の栄養.csv",index_col=0)

In [7]:
df_max_min 

,min,max
栄養名,,
タンパク質,50.0,NaN
脂質,NaN,45.0
炭水化物,NaN,NaN
ナトリウム,NaN,NaN
カリウム,2000.0,NaN
カルシウム,650.0,NaN
リン,800.0,3000.0
鉄,10.5,40.0
ビタミンA,700.0,2700.0


# 低カロリー問題

In [8]:
# 変数の設定
df["個数"] = pulp.LpVariable.dicts('個数',df.index,lowBound=0,upBound=3,cat='Integer').values()

In [9]:
# 最小化問題
problem = pulp.LpProblem('完全栄養マクドナルド問題',sense=pulp.LpMinimize)

In [10]:
# 目的変数の設定
problem.setObjective(pulp.lpDot(df["エネルギー"],df["個数"]))

In [11]:
# 拘束条件
for nutrition in df_max_min.index:
    if not np.isnan(df_max_min.loc[nutrition,"max"]):
      problem.addConstraint(pulp.lpDot(df[nutrition],df["個数"])<=df_max_min.loc[nutrition,"max"])
    if not np.isnan(df_max_min.loc[nutrition,"min"]):
      problem.addConstraint(pulp.lpDot(df[nutrition],df["個数"])>=df_max_min.loc[nutrition,"min"])

# 重量に条件
problem.addConstraint(pulp.lpDot(df["製品重量"],df["個数"])<=2000)

In [12]:
# 問題を解く
status = problem.solve()
print(pulp.LpStatus[problem.status])

Optimal


In [13]:
# 答えの表示
for v in problem.variables():
  if v.varValue != 0:
    print(f'{v.name} = {v.varValue}')

print(f"総カロリー:{pulp.value(problem.objective)}kcal")

個数_えだまめコーン = 3.0
個数_りんご&クリーム = 1.0
個数_ケチャップ = 1.0
個数_サイドサラダ = 3.0
個数_シャカチキ_チェダーチーズ味シーズニング = 1.0
個数_シャカチキ_レッドペッパー味シーズニング = 2.0
個数_プチパンケーキ = 1.0
個数_プレミアムローストコーヒー(S) = 3.0
個数_マックグリドル_ベーコンエッグ = 1.0
個数_マックシェイク?_チョコレート(M) = 1.0
個数_マックシェイク?_チョコレート(S) = 1.0
個数_野菜生活100(S) = 1.0
総カロリー:1604.0kcal


In [14]:
for nutrition in df_max_min.index:
  print(nutrition + ":" + str(round(pulp.lpDot(df[nutrition], df["個数"]).value(),ndigits=1)))

タンパク質:52.1
脂質:43.3
炭水化物:251.6
ナトリウム:2610.0
カリウム:2915.0
カルシウム:816.0
リン:1345.0
鉄:10.5
ビタミンA:770.0
ビタミンB1:1.3
ビタミンB2:1.4
ナイアシン:12.0
ビタミンC:102.0
コレステロール:247.0
食物繊維:18.4
食塩相当量:6.5
